In [31]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer, LoggingHandler, util, evaluation, models, InputExample
import logging
import os
import gzip
import csv
import random
import torch
from collections import Counter


In [7]:
# Model for which we apply dimensionality reduction
model = SentenceTransformer("all-MiniLM-L6-v2")
# New size for the embeddings
new_dimension = 32


config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 71.4kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 4.05MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 27.8kB/s]
config.json: 100%|██████████| 612/612 [00:00<00:00, 270kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:02<00:00, 31.0MB/s]
/Users/pablo/.conda/envs/microsoft_sepsis/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 159kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 4.80MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 13.3MB/s]
1_Pooling/config.js

In [40]:
noteevents = pd.read_csv("noteevents_preprocessed_text_granular_label.csv")
lab = pd.read_csv("patients_no_text_sepsis_labels.csv")


In [42]:
hours = pd.read_csv("lab_hours.csv")
hours

,Unnamed: 0,hour
0,0,0.0
1,1,1.0
2,2,2.0
3,3,3.0
4,4,4.0
...,...,...
1991934,1991934,21.0
1991935,1991935,22.0
1991936,1991936,23.0
1991937,1991937,24.0


In [43]:
lab["hour"] = hours["hour"]
lab

,Unnamed: 0.1,Unnamed: 0,a:action,m:charttime,m:icustayid,m:presumed_onset,o:Arterial_BE,o:Arterial_lactate,o:Arterial_pH,o:BUN,...,o:output_4hourly,o:output_total,o:paCO2,o:paO2,o:re_admission,r:reward,step,traj,sepsis_label,hour
0,0,0,10,7.245486e+09,200003.0,7.245590e+09,1.071661,-0.560359,1.069870,-0.380498,...,0.635895,0.727866,-0.374712,-0.689066,-0.5,0.0,0,1,1,0.0
1,1,1,10,7.245490e+09,200003.0,7.245590e+09,1.071661,-0.560359,1.069870,-0.380498,...,0.844605,0.732548,-0.374712,-0.689066,-0.5,0.0,1,1,1,1.0
2,2,2,10,7.245493e+09,200003.0,7.245590e+09,1.071661,-0.560359,1.069870,-0.380498,...,1.021651,0.740802,-0.374712,-0.689066,-0.5,0.0,2,1,1,2.0
3,3,3,10,7.245497e+09,200003.0,7.245590e+09,1.071661,-0.560359,1.069870,-0.380498,...,0.986171,0.747959,-0.374712,-0.689066,-0.5,0.0,3,1,1,3.0
4,4,4,10,7.245500e+09,200003.0,7.245590e+09,1.071661,-0.560359,1.069870,-0.380498,...,0.690782,0.750601,-0.374712,-0.689066,-0.5,0.0,4,1,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991934,1991934,1991934,5,4.659939e+09,299999.0,0.000000e+00,-0.019540,-0.744383,-0.112773,-0.519833,...,NaN,NaN,-0.076793,-0.483898,-0.5,0.0,20,24402,0,21.0
1991935,1991935,1991935,0,4.659942e+09,299999.0,0.000000e+00,-0.019540,-0.744383,-0.112773,-0.519833,...,NaN,NaN,-0.076793,-0.483898,-0.5,0.0,21,24402,0,22.0
1991936,1991936,1991936,0,4.659946e+09,299999.0,0.000000e+00,-0.019540,-0.744383,-0.112773,-0.519833,...,NaN,NaN,-0.076793,-0.483898,-0.5,0.0,22,24402,0,23.0
1991937,1991937,1991937,0,4.659950e+09,299999.0,0.000000e+00,-0.019540,-0.744383,-0.112773,-0.519833,...,NaN,NaN,-0.076793,-0.483898,-0.5,0.0,23,24402,0,24.0


In [45]:
noteevents.dropna(subset=['clean_text'], inplace=True)

In [5]:
sentences = list(noteevents["clean_text"])
sentences

['abdomen contrast pelvis contrast clip clip number radiology reason hypotension severe abd pain distention please quick field view ______________________________________________________________________________ hospital medical condition year old man cad gib reason examination hypotension severe abd pain distention please quick look non contrast abd contraindications contrast ______________________________________________________________________________ wet read first name namepattern doctor first name aaa diffuse vasc calcifications ______________________________________________________________________________ final report history severe abdominal pain hypotension please assess aorta non contrast scan technique helically acquired contiguous axial images obtained abdomen pelvis without administration oral contrast per request emergency room attending physician caring patient abdomen without contrast incompletely imaged coronary artery calcifications dependent atelectasis within visuali

In [8]:
random.shuffle(sentences)
# To determine the PCA matrix, we need some example sentence embeddings.
# Here, we compute the embeddings for 20k random sentences from the AllNLI dataset
pca_train_sentences = sentences[0:20000]
train_embeddings = model.encode(pca_train_sentences, convert_to_numpy=True)

In [9]:
# Compute PCA on the train embeddings matrix
pca = PCA(n_components=new_dimension)
pca.fit(train_embeddings)
pca_comp = np.asarray(pca.components_)

In [10]:
# We add a dense layer to the model, so that it will produce directly embeddings with the new size
dense = models.Dense(
    in_features=model.get_sentence_embedding_dimension(),
    out_features=new_dimension,
    bias=False,
    activation_function=torch.nn.Identity(),
)
dense.linear.weight = torch.nn.Parameter(torch.tensor(pca_comp))
model.add_module("dense", dense)


In [14]:
sentence_arr = noteevents["clean_text"]
test = list(sentence_arr[:100])
test

['abdomen contrast pelvis contrast clip clip number radiology reason hypotension severe abd pain distention please quick field view ______________________________________________________________________________ hospital medical condition year old man cad gib reason examination hypotension severe abd pain distention please quick look non contrast abd contraindications contrast ______________________________________________________________________________ wet read first name namepattern doctor first name aaa diffuse vasc calcifications ______________________________________________________________________________ final report history severe abdominal pain hypotension please assess aorta non contrast scan technique helically acquired contiguous axial images obtained abdomen pelvis without administration oral contrast per request emergency room attending physician caring patient abdomen without contrast incompletely imaged coronary artery calcifications dependent atelectasis within visuali

In [15]:
# Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(test)
# Print the embeddings
for sentence, embedding in zip(test, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")


Sentence: abdomen contrast pelvis contrast clip clip number radiology reason hypotension severe abd pain distention please quick field view ______________________________________________________________________________ hospital medical condition year old man cad gib reason examination hypotension severe abd pain distention please quick look non contrast abd contraindications contrast ______________________________________________________________________________ wet read first name namepattern doctor first name aaa diffuse vasc calcifications ______________________________________________________________________________ final report history severe abdominal pain hypotension please assess aorta non contrast scan technique helically acquired contiguous axial images obtained abdomen pelvis without administration oral contrast per request emergency room attending physician caring patient abdomen without contrast incompletely imaged coronary artery calcifications dependent atelectasis within

In [17]:
full_sentences = list(noteevents["clean_text"])
sentence_embeddings = model.encode(full_sentences)


In [46]:
#combine text and sBert embeddings
temp = noteevents.copy()
x = pd.DataFrame(sentence_embeddings, index=temp.index).add_prefix('sBert:')
x
h = pd.concat([temp, x], axis=1)
ll = h.drop(columns=['sepsis_label', 'm:presumed_onset',
       'granular_label', 'Unnamed: 0', 'Unnamed: 0.1','subject_id', 'hadm_id', 'category',
       'description', 'iserror', 'text', 'charttime', 'storetime',])
df_ = lab.loc[lab["m:icustayid"].isin(np.unique(ll["m:icustayid"]))]
print(len(np.unique(ll["m:icustayid"])), len(np.unique(df_["m:icustayid"])))

41774 41774


In [19]:
x

,sBert:0,sBert:1,sBert:2,sBert:3,sBert:4,sBert:5,sBert:6,sBert:7,sBert:8,sBert:9,...,sBert:22,sBert:23,sBert:24,sBert:25,sBert:26,sBert:27,sBert:28,sBert:29,sBert:30,sBert:31
0,-0.507864,0.235520,0.017482,-0.125230,0.036280,-0.062636,-0.051188,-0.313510,-0.108823,-0.159074,...,-0.017881,-0.115798,-0.197134,0.007230,0.070957,0.041549,-0.040260,-0.125860,0.064280,-0.134453
1,-0.451370,0.336454,0.263941,0.271171,-0.104731,0.052452,-0.152813,-0.385071,0.011239,0.039794,...,-0.003455,-0.091406,-0.108700,-0.008616,-0.058711,0.018290,-0.108652,-0.034918,0.019815,-0.005620
2,-0.545362,0.318873,0.112448,0.159560,-0.115543,0.037398,-0.154526,-0.361784,-0.060006,0.031074,...,-0.004942,0.047419,-0.088980,-0.089459,-0.035275,-0.010772,-0.030314,-0.085824,0.040106,-0.026809
3,-0.520119,0.323225,0.151977,0.163492,-0.136283,0.047496,-0.165475,-0.379649,-0.111925,0.077005,...,0.039113,-0.010790,-0.077177,-0.103884,0.030968,-0.051167,-0.113341,-0.021931,0.006752,-0.032217
4,-0.529784,0.332215,0.142704,0.153253,-0.088160,0.052918,-0.193467,-0.376044,-0.118891,0.042613,...,0.006870,-0.027884,-0.102637,-0.105707,-0.013194,-0.046902,-0.082788,-0.057701,0.022160,-0.030884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311535,-0.510889,-0.256551,0.225859,0.168554,0.111202,-0.102978,0.044406,-0.191986,-0.120544,0.071235,...,-0.010237,-0.041012,-0.189417,-0.053977,-0.007118,0.030475,-0.054567,-0.029284,0.020328,-0.077089
1311536,-0.457990,-0.258802,0.147758,0.154789,0.061958,-0.099401,0.041263,-0.255316,-0.052624,0.036934,...,-0.000994,-0.048638,-0.173146,-0.065835,-0.058786,-0.024382,-0.071238,-0.054179,0.069972,-0.060134
1311537,-0.514014,-0.206679,0.286752,-0.010555,0.174957,-0.001933,0.064754,-0.212337,-0.266043,0.040974,...,-0.054263,-0.079169,-0.157433,0.017757,-0.030816,-0.020133,-0.038152,-0.052084,0.074139,-0.055182
1311538,0.438971,0.006395,0.066345,0.046452,-0.125374,-0.318873,-0.216974,-0.297142,-0.165709,-0.162102,...,-0.088197,0.069815,-0.089321,-0.036223,-0.002624,-0.009523,-0.072992,-0.037488,0.024453,-0.098065


In [47]:
DF1 = df_.sort_values(by="m:charttime")
DF2 = ll.sort_values(by="chartdate")
DF2.rename(columns={"chartdate":"m:charttime"}, inplace=True)
DF1["m:icustayid"] = DF1["m:icustayid"].astype(int)
DF2["m:icustayid"] = DF2["m:icustayid"].astype(int)
final_df = pd.merge_asof(DF1, DF2, on="m:charttime", by="m:icustayid", direction='nearest')    

In [48]:
final_df[["m:icustayid", "hour"]]

,m:icustayid,hour
0,202134,0.0
1,202134,3.0
2,202134,14.0
3,202134,15.0
4,202134,16.0
...,...,...
1978153,234115,74.0
1978154,234115,75.0
1978155,234115,76.0
1978156,234115,77.0


In [49]:
final_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'a:action', 'm:charttime', 'm:icustayid',
       'm:presumed_onset', 'o:Arterial_BE', 'o:Arterial_lactate',
       'o:Arterial_pH', 'o:BUN', 'o:Calcium', 'o:Chloride', 'o:Creatinine',
       'o:DiaBP', 'o:FiO2_1', 'o:GCS', 'o:Glucose', 'o:HCO3', 'o:HR', 'o:Hb',
       'o:INR', 'o:Magnesium', 'o:MeanBP', 'o:PT', 'o:PTT', 'o:PaO2_FiO2',
       'o:Platelets_count', 'o:Potassium', 'o:RR', 'o:SGOT', 'o:SGPT',
       'o:SIRS', 'o:SOFA', 'o:Shock_Index', 'o:Sodium', 'o:SpO2', 'o:SysBP',
       'o:Temp_C', 'o:Total_bili', 'o:WBC_count', 'o:Weight_kg', 'o:age',
       'o:cumulated_balance', 'o:gender', 'o:input_4hourly', 'o:input_total',
       'o:max_dose_vaso', 'o:mechvent', 'o:output_4hourly', 'o:output_total',
       'o:paCO2', 'o:paO2', 'o:re_admission', 'r:reward', 'step', 'traj',
       'sepsis_label', 'hour', 'clean_text', 'sBert:0', 'sBert:1', 'sBert:2',
       'sBert:3', 'sBert:4', 'sBert:5', 'sBert:6', 'sBert:7', 'sBert:8',
       'sBert:9', 'sBer

In [50]:
final_df.to_csv("lab_sBert_concat_fulldata.csv")